In [1]:
import sqlite3
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re  # 用于字符串的正则匹配

# 配置Chrome（可选“无头模式”，即无界面运行）
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 取消注释则隐藏浏览器界面
driver = webdriver.Chrome(options=chrome_options)

There was an error managing chromedriver (error decoding response body); using driver found in the cache


In [15]:

#URL1 ="https://esf.fang.com/house/kw%B1%B1%CC%AB%C6%BD%D7%AF/" # 房天下网站(北太平庄）
#URL2 = "https://zu.fang.com/house-a015277-b05/s31/" # 房天下租房网站（北太平庄）

url1 = "https://esf.fang.com/house/kw%B1%B1%CC%AB%C6%BD%D7%AF/" # 房天下二手房URL（需确保编码/地址正确）
driver.get(url1)
esf_data = []  # 存储所有页的房源数据

for page in range(5):  # 循环处理“前五页”，range(5)对应页码 0~4
    try:
        # 1. 等待当前页“房源列表”加载完成
        wait = WebDriverWait(driver, 100)
        esf_houses = wait.until(
            EC.presence_of_all_elements_located(
                (By.XPATH, "//div[@class='shop_list shop_list_4']/dl")  # 需根据**实际网页结构**调整XPath
            )
        )

        # 2. 遍历当前页的每一套房源，提取信息
        for house in esf_houses:
            try:
                # 提取“标题”（相对路径：在当前house内查找）
                title = house.find_element(By.XPATH, ".//span[@class='tit_shop']").text
                # 提取“总价”（相对路径：在当前house内查找）
                total_price = house.find_element(By.XPATH, ".//span[@class='red']").text
                # 提取“单价”（相对路径：在当前house内查找）
                unit_price = house.find_element(By.XPATH, ".//dd[@class='price_right']/span[2]").text

                # 将单套房源数据存入列表
                esf_data.append({
                    "title": title,
                    "total_price": total_price,
                    "unit_price": unit_price,
                    "type": "二手房"
                })
            except Exception as e:
                print(f"提取单套房源出错：{e}")

        # 3. 点击“下一页”按钮（需根据**实际网页结构**调整XPath）
        next_page_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//a[text()='下一页']")  # 示例XPath，需替换为实际“下一页”按钮的定位方式
            )
        )
        next_page_btn.click()

        # （可选）等待下一页加载完成（确保元素更新）
        wait.until(EC.staleness_of(esf_houses[0]))  # 等待“当前页房源列表”失效，说明页面在跳转

    except Exception as e:
        print(f"第{page+1}页处理出错：{e}")
        break  # 若某一页出错，可选择中断循环（也可改为continue继续下一页）

# 4. 处理最终爬取的数据
if esf_data:
    df = pd.DataFrame(esf_data)
    display(df.head(120))  # 在Jupyter Notebook/Lab中“美观展示”DataFrame
else:
    print("未爬取到二手房数据")

,title,total_price,unit_price,type
0,德胜门·新街口外大街甲8号院·3室·1厅,990万,121472元/㎡,二手房
1,央企社区 诚意出售 新街口外大街甲12号院 3室 1厅,780万,136842元/㎡,二手房
2,(客户0)花园路10号院 3室1厅 57.4平 南北通透,406万,70731元/㎡,二手房
3,"西城德胜,马甸南村,南北两居",520万,87394元/㎡,二手房
4,西城 德胜门 马甸新风北里 南北三居,720万,105571元/㎡,二手房
...,...,...,...,...
115,德胜· 马甸南村 · 满五无在读 看房方便 靠谱卖,710万,132710元/㎡,二手房
116,北太平庄·北影小区·2室·1厅,630万,69459元/㎡,二手房
117,西城德胜 新上中直社区大客厅两居室 满五年格局好,929万,135422元/㎡,二手房
118,延安大院 新上三居 满五公房 央产已经办理 随时签约,1150万,128062元/㎡,二手房


In [17]:

# 目标租房页面 URL
url2 = "https://zu.fang.com/house-ag15277-bd5-s31/"  
driver.get(url2)  # 打开页面
rent_data = []     # 存储所有页的租房数据
max_pages = 5      # 爬取“前五页”（页码 0~4）

for page in range(max_pages):
    try:
        # -------- 1. 等待“当前页租房列表”加载 --------
        rent_houses = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                # 定位所有包含租房信息的 dl 条目（需根据实际页面调整 XPath）
                (By.XPATH, "//div[@id='houselistbody']//dl[contains(@class, 'list')]")
            )
        )

        # -------- 2. 遍历“当前页每套租房”，提取信息 --------
        for house in rent_houses:
            try:
                # 提取“标题”（相对路径：当前房源条内查找）
                title_element = house.find_element(By.XPATH, ".//p[@class='title']/a")
                title = title_element.text.strip() if title_element else "暂无标题"

                # 提取“总租金”
                rent_message_element = house.find_element(By.XPATH, ".//p[@class='font15 mt12 bold']")
                rent_message = rent_message_element.text.strip() if rent_message_element else "信息待定"
                area_match = re.search(r'(\d+)㎡', rent_message)  # 匹配“123㎡”这样的格式
                area = area_match.group(1) + "㎡" if area_match else "面积未知"  # 提取到则保留，否则设为默认值

                # 提取“单位租金”
                unit_rent_element = house.find_element(By.XPATH, ".//p[@class='mt5 alingC']/span[@class='price']")
                unit_rent = (unit_rent_element.text.strip() + " 元/月") if unit_rent_element else "价格待定"

                # 存入单套房源数据
                rent_data.append({
                    "title": title,
                    "rent_message": rent_message,
                    "unit_rent": unit_rent,
                    "area":area,
                    "type": "租房"
                })
            except Exception as e:
                print(f"第{page+1}页：提取单套租房数据失败 → {e}")
                # 继续处理下一个房源（异常不中断循环）

        # -------- 3. 点击“下一页”按钮，进入下一页 --------
        next_page_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                # 定位“下一页”按钮（需根据实际页面调整 XPath）
                (By.XPATH, "//div[contains(@class, 'fanye')]//a[text()='下一页']")
            )
        )
        next_page_btn.click()

        # （可选）等待下一页加载完成（通过“当前页房源列表失效”判断）
        WebDriverWait(driver, 10).until(
            EC.staleness_of(rent_houses[0])
        )

    except Exception as e:
        print(f"第{page+1}页：整体处理失败 → {e}")
        break  # 某页出错时中断循环（也可改为 continue 继续下一页）

# -------- 4. 处理并展示最终爬取的数据 --------
if rent_data:
    df = pd.DataFrame(rent_data)
    display(df)  # Jupyter 中“美观展示”DataFrame
else:
    print("未爬取到租房数据")

# 关闭浏览器（若需主动关闭，取消下方注释）
# driver.quit()

,title,rent_message,unit_rent,area,type
0,远大园三区 4室2卫2厅 204平简装修 22000,整租|4室2厅|204㎡|朝南北,22000 元/月,204㎡,租房
1,新出位置安静 可直接入住 楼层好 330平米 三面采,整租|3室3厅|330㎡|朝南北,138000 元/月,330㎡,租房
2,15套起看 媲美万柳书院万城华府玺园 稳定出租,整租|4室2厅|416㎡|朝东,80000 元/月,416㎡,租房
3,10套可看万城华府尚园 新出平层4居室 带车库 同看,整租|5室3厅|468㎡|朝东,58000 元/月,468㎡,租房
4,万城华府海园 5室3厅4卫 视野开阔 主卧朝南,整租|5室3厅|381㎡|朝南北,120000 元/月,381㎡,租房
...,...,...,...,...,...
295,海淀钓鱼台七号院 西三环金融街 大平层300-850,整租|5室3厅|593㎡|朝南北,115000 元/月,593㎡,租房
296,"泛海南北通透,户型好,保养如新,家具电器齐全,地暖,",整租|3室2厅|188㎡|朝南北,27000 元/月,188㎡,租房
297,东城区南锣鼓巷鼓楼交道口会所接待四合院出租 带KTV,整租|5室3厅|320㎡|朝南北,90000 元/月,320㎡,租房
298,东城区 安定门鼓楼国子监 接待会所宴请 门前一车位四,整租|8室2厅|240㎡|朝南,80000 元/月,240㎡,租房


In [24]:
url2 = "https://zu.fang.com/house-a@15277-b05/s31/"  # 房天下租房URL
driver.get(url2)

# 等待租房列表加载。关键：使用更稳定的XPath
try:
    rent_houses = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located(
            # 定位所有包含租房信息的dl条目
            (By.XPATH, "//div[@class='houseList']//dl[@class='list hiddenMap rel']")
        )
    )
except Exception as e:
    print(f"等待租房列表加载时出错: {e}")
    rent_houses = [] # 避免后续循环出错
# 提取每套租房的信息
rent_data = []
for house in rent_houses:
    try:
        # 使用相对路径（以.开头）在当前条目内查找
        # 标题
        title_element = house.find_element(By.XPATH, ".//p[@class='title']/a")
        title = title_element.text.strip() if title_element else "暂无标题"
        
        # 总租金
        rent_message_element = house.find_element(By.XPATH, ".//p[@class='font15 mt12 bold']")
        rent_message = rent_message_element.text.strip() if rent_message_element else "价格待定"
        area_match = re.search(r'(\d+)㎡', rent_message)  # 匹配“123㎡”这样的格式
        area = area_match.group(1) + "㎡" if area_match else "面积未知"  # 提取到则保留，否则设为默认值
        # 单位租金 (注意：当前测试页面此元素可能不存在或结构不同)
        # 先定位到存放详情的p标签
        #detail_p = house.find_element(By.XPATH, ".//p[@class='mt12']")
        # 从文本中提取包含“㎡”的部分（处理可能的空格或格式问题）
        # 从文本中提取包含“㎡”的部分（处理可能的空格或格式问题）
        #area_parts = [text.strip() for text in detail_p.text.split("|") if "㎡" in text]
        # 列表非空时取第一个元素，否则给默认值
        #unit_rent = area_parts[0] if area_parts else "面积信息未获取到"
        #print("面积：", unit_mianji)
        unit_rent_element = house.find_element(By.XPATH, ".//p[@class='mt5 alingC']/span[@class='price']")
        unit_rent =(unit_rent_element.text.strip()+"元/月") if rent_message_element else "价格待定"
        rent_data.append({
            "title": title,
            "rent_message": rent_message,
            "unit_rent": unit_rent,
            "type": "租房",
            "area":area
        })
    except Exception as e:
        print(f"提取单个租房信息时出错: {e}")
        # 可以选择继续处理下一个房源

# 关闭浏览器
driver.quit()

# 展示数据
if rent_data:
    df = pd.DataFrame(rent_data)
    try:
        from IPython.display import display
        display(df)
    except ImportError:
        print(df)
else:
    print("未爬取到租房数据")

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=141.0.7390.66); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff76e56e955+80021]
	GetHandleVerifier [0x0x7ff76e56e9b0+80112]
	(No symbol) [0x0x7ff76e2f060f]
	(No symbol) [0x0x7ff76e2dc145]
	(No symbol) [0x0x7ff76e30177a]
	(No symbol) [0x0x7ff76e378b06]
	(No symbol) [0x0x7ff76e398fa2]
	(No symbol) [0x0x7ff76e371003]
	(No symbol) [0x0x7ff76e3395d1]
	(No symbol) [0x0x7ff76e33a3f3]
	GetHandleVerifier [0x0x7ff76e82dd0d+2960461]
	GetHandleVerifier [0x0x7ff76e827fca+2936586]
	GetHandleVerifier [0x0x7ff76e848a07+3070279]
	GetHandleVerifier [0x0x7ff76e58843e+185214]
	GetHandleVerifier [0x0x7ff76e58fe8f+216527]
	GetHandleVerifier [0x0x7ff76e577b94+117460]
	GetHandleVerifier [0x0x7ff7

In [19]:
def save_to_database(esf_data, rent_data):
    # 1. 建立数据库连接（若文件不存在则自动创建）
    # 示例：保存到D盘的“房源数据”文件夹（需先手动创建该文件夹）
    conn = sqlite3.connect('D:\人工智能\房源数据\house.db')
    cursor = conn.cursor()
    
    try:
        # 2. 先删除旧表（解决表结构不更新的问题）
        cursor.execute("DROP TABLE IF EXISTS house_info;")
        conn.commit()  # 提交删除操作
        
        # 3. 重新创建表（确保包含type列，且列名与插入语句一致）
        cursor.execute('''
        CREATE TABLE house_info (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,           
            total_price TEXT,     
            unit_price TEXT,     
            rent_message TEXT,     
            unit_rent TEXT,    
            area TEXT,
            type TEXT             
        )
        ''')
        conn.commit()  # 提交建表操作
        
        # 4. 插入二手房数据
        for data in esf_data:
            cursor.execute('''
            INSERT INTO house_info (
                title, total_price, unit_price, 
                rent_message, unit_rent, area, type
            ) VALUES (?, ?, ?, ?, ?, ?,?)
            ''', (
                data["title"],
                data["total_price"],
                data["unit_price"],
                None,  
                None,
                None,
                data["type"]
            ))
        
        # 5. 插入租房数据
        for data in rent_data:
            cursor.execute('''
            INSERT INTO house_info (
                title, total_price, unit_price, 
                rent_message, unit_rent, area, type
            ) VALUES (?, ?, ?, ?, ?, ?,?)
            ''', (
                data["title"],
                None, 
                None,  
                data["rent_message"], 
                data["unit_rent"],
                data["area"],
                data["type"]
            ))
        
        conn.commit()  # 提交所有插入操作
        print("数据成功写入数据库！")
    
    except sqlite3.OperationalError as e:
        print(f"数据库操作错误：{e}")
        conn.rollback()  # 出错时回滚事务
    finally:
        # 无论成功与否，都关闭连接
        conn.close()
        print("数据库连接已关闭")

# 执行数据存储
save_to_database(esf_data, rent_data)

数据成功写入数据库！
数据库连接已关闭


In [20]:

# 1. 连接数据库（与存储时的文件名一致）
conn = sqlite3.connect('D:\人工智能\房源数据\house.db')  # 替换为你的数据库文件名
cursor = conn.cursor()

# 3. 查询表中所有数据（以表名 house_info 为例，需替换为实际表名）
table_name = "house_info"  # 替换为你实际的表名
cursor.execute(f"SELECT * FROM {table_name};")
rows = cursor.fetchall()

# 4. 打印列名和数据
columns = [desc[0] for desc in cursor.description]
print("列名：", columns)
for row in rows:
    print(row)

# 5. 关闭连接
conn.close()

列名： ['id', 'title', 'total_price', 'unit_price', 'rent_message', 'unit_rent', 'area', 'type']
(1, '德胜门·新街口外大街甲8号院·3室·1厅', '990万', '121472元/㎡', None, None, None, '二手房')
(2, '央企社区 诚意出售 新街口外大街甲12号院 3室 1厅', '780万', '136842元/㎡', None, None, None, '二手房')
(3, '(客户0)花园路10号院 3室1厅 57.4平 南北通透', '406万', '70731元/㎡', None, None, None, '二手房')
(4, '西城德胜,马甸南村,南北两居', '520万', '87394元/㎡', None, None, None, '二手房')
(5, '西城 德胜门 马甸新风北里 南北三居', '720万', '105571元/㎡', None, None, None, '二手房')
(6, '德胜门 · 马甸南村 · 3室 · 1厅', '1100万', '138888元/㎡', None, None, None, '二手房')
(7, '1997年月季园通透两居带电梯两居户型方正税费少看房有钥匙', '378万', '65842元/㎡', None, None, None, '二手房')
(8, '牡丹园 · 花园公寓 · 1室 · 1厅', '530万', '80940元/㎡', None, None, None, '二手房')
(9, '央企社区 满五年一套 中间层 产权清晰 小区管理好', '780万', '136842元/㎡', None, None, None, '二手房')
(10, '牡丹园马甸冠城园 满五V1 高层景观三居 靠谱签约 看房方便', '1390万', '93413元/㎡', None, None, None, '二手房')
(11, '有正规大客厅,马甸德胜新,95年楼龄,正对公园 ,户型方正', '660万', '137214元/㎡', None, None, None, '二手房')
(12, '西城德胜,高楼层精装修两居室,采光视野很棒哟!', '622万', '104187元/㎡', 